<a href="https://colab.research.google.com/github/Efferescent/Assist.org-Equivalent-Classes/blob/main/Assist_org_Equivalent_Class_Search_Class_Search_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
College_Code_Search = str(input("Please enter your college code (i.e. CPSLO): ", ))
Major_Input = str(input("Please enter your major (i.e. COMPUTER SCIENCE, B.S.): "))
sv_1 = str(input('Class Code (i.e. CSC): ', ))
sv_1_plus = '\u200b'
sv_2 = str(input('Class Code Number (i.e. 202): ', ))
sv_2_plus = ' - \u200b'
sv_3 = str(input('Class Name and Unit (i.e. Data Structures (4.00)): ', ))
search_value = str(sv_1 + sv_1_plus + ' ' + sv_2 + sv_2_plus + sv_3)
number_value = int(input("\nIf on Assist.org, the class has a same as class (i.e. Same as: CPE 202), put in 3 for this. If not, put in 2. ", ))
academic_year = str(input('Put in count of year after 1950(i.e. 2023 is 73): ',))


!pip install PyMuPDF
import requests
import pandas as pd
import warnings
import os
import fitz

warnings.filterwarnings('ignore')

response = requests.get('https://assist.org/api/institutions')
institutions = response.json()

df_institution = pd.DataFrame(data = institutions)
drop_list = ['names','prefers2016LegacyReport','isCommunityCollege','category','termType','termTypeAcademicYears']
df_institution.drop(drop_list,axis=1,inplace = True)

df_institution = df_institution[df_institution['code'].str.contains(College_Code_Search)]
College_ID = str(df_institution['id'].tolist())
College_ID_Modified = College_ID[1:-1]

response_agreements = requests.get('https://assist.org/api/institutions/'+ College_ID_Modified+ '/agreements')
agreements = response_agreements.json()

df_agreements = pd.DataFrame(data = agreements)
df_agreements.sort_values('institutionParentId',ascending = False,inplace = True)
df_agreements.drop_duplicates('institutionParentId',inplace = True)

CC_Filter = df_agreements['isCommunityCollege'] == True
CC_DF = df_agreements[CC_Filter]
CC_List = CC_DF['institutionParentId'].tolist()

df_Reports = pd.DataFrame(data = ())

for cc in CC_List:
  url = 'https://assist.org/api/agreements?receivingInstitutionId=11&sendingInstitutionId='+str(cc)+'&academicYearId='+academic_year+'&categoryCode=major'
  response_reports = requests.get(url)
  Articulation = response_reports.json()
  df_Reports = df_Reports.append(Articulation['reports'])

Major_Filter = df_Reports['label'] == str(Major_Input)
Major_DF = df_Reports[Major_Filter]
Major_DF.drop('ownerInstitutionId',axis=1,inplace = True)
Major_DF = Major_DF.drop_duplicates()

Major_DF_List = Major_DF['key'].tolist()

urls = []

for key in Major_DF_List:
  urls.append('https://assist.org/api/artifacts/'+str(key))

output_dir = '.'

Name_PDF_List = []

for url in urls:
  response = requests.get(url)
  if response.status_code == 200:
    name_pdf = url+'.pdf'
    file_path=os.path.join(output_dir, os.path.basename(name_pdf))
    Name_PDF_List.append(name_pdf[-12:])
    with open(file_path, 'wb') as f:
      f.write(response.content)

Equivalent_Class_List = []

for pdf_path in Name_PDF_List:
    doc = fitz.open(pdf_path)

    for page_num in range(doc.page_count):
        page = doc[page_num]
        text = page.get_text()
        Text_List = text.split('\n')
        df6 = pd.DataFrame({'Class and Articulation': Text_List})
        index_value = df6.index[df6['Class and Articulation'] == search_value].tolist()

        if index_value:
            index_integer = int(index_value[0])
            if index_integer + number_value < len(df6):
                third_index_after_target = df6.loc[index_integer + number_value, 'Class and Articulation']
                modified_string = third_index_after_target.replace('b', '').replace('B', '').replace('/u', '')
                Equivalent_Class_List.append(modified_string)
            else:
                print("Index out of bounds.")

df_equivalent_class = pd.DataFrame(data = ())
df_equivalent_class['Equivalent Classes'] = Equivalent_Class_List
df_equivalent_class_filter = df_equivalent_class['Equivalent Classes'] != 'No Course Articulated'
df_with_equivalent_class_filter = df_equivalent_class[df_equivalent_class_filter]
df_with_equivalent_class_filter.set_index('Equivalent Classes')
df_with_equivalent_class_filter = df_with_equivalent_class_filter.drop_duplicates()

print('\n')
print("The class you have chosen is: " + search_value)
print('\n')

df_with_equivalent_class_filter

Please enter your college code (i.e. CPSLO): CPSLO
Please enter your major (i.e. COMPUTER SCIENCE, B.S.): INDUSTRIAL ENGINEERING, B.S.
Class Code (i.e. CSC): CPE
Class Code Number (i.e. 202): 101
Class Name and Unit (i.e. Data Structures (4.00)): Fundamentals of Computer Science (4.00)

If on Assist.org, the class has a same as class (i.e. Same as: CPE 202), put in 3 for this. If not, put in 2. 3
Put in count of year after 1950(i.e. 2023 is 73): 73


The class you have chosen is: CPE​ 101 - ​Fundamentals of Computer Science (4.00)




,Equivalent Classes
0,ENGR ​ 40 - ​Programming for Scientists and En...
2,ENGR​ 40 - ​Programming for Scientists and Eng...
3,CIS​ 17A - ​Programming Concepts and Methodolo...
4,CIS​ 17A - ​Programming Concepts and Methodolo...
7,CISP​ 360 - ​Introduction to Structured Progra...
...,...
110,CSCI​ 001 - ​Programming Concepts/Method 1 (4.00)
111,CIS​ 278 - ​(CS1)Programming Methods I: C++ (4...
112,COMP​ 232 - ​Advanced Programming in JAVA (4.00)
113,CS​ 116 - ​Programming In C++ (3.00)
